chuẩn bị dữ liệu

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot as plt

# Tải dữ liệu MNIST và chia thành tập huấn luyện và tập kiểm tra
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Chuẩn hóa dữ liệu và chuyển đổi nhãn thành one-hot encoding
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

xử lý dữ liệu

In [2]:
#flatten train and test img
train_images = train_images.reshape(60000,784)
print(train_images.shape)
test_images = test_images.reshape(10000,784)
print(test_images.shape)

(60000, 784)
(10000, 784)


hàm dự đoán

In [3]:
def predict(test_image,q):
    min = np.linalg.norm(q[0,:]-test_image)
    index =0
    for i in range(10):
        min1=np.linalg.norm(q[i,:]-test_image)
        if min>min1:
            index = i
            min = min1
    return index

hàm tính độ chính xác

In [4]:
def accuracy(set_images,set_labels,q_table):
    number =0
    for i in range(set_images.shape[0]):
        image = set_images[i]
        label = np.argmax(set_labels[i])
        predicted_label= predict(image,q_table) 
        if predicted_label == label: number+=1
    print(number)
    return (float(number / set_images.shape[0]))*100

hàm cập nhật

In [5]:
def update_q(q,set_images,set_labels,rate_learning=1e-10):
    count = 0
    for i in range(60000):
        trueLable = np.argmax(set_labels[i])
        preLable = predict(set_images[i],q)
        # print(preLable)
        if trueLable == preLable:
            q[trueLable,:]= q[trueLable,:] +  set_images[i]*rate_learning
            count +=1
        else: 
            q[trueLable,:]= q[trueLable,:] +  set_images[i]*rate_learning*100
            q[preLable,:]= q[preLable,:] -  set_images[i]*rate_learning*100
            for i in range(10):
                if i!=trueLable and i !=preLable:
                    q[i,:]= q[i,:] -  set_images[i]*rate_learning*0.001
    return count,q

train

In [7]:
q = np.zeros([10,784])
print(q.shape)
max = 0
qTemp=qMax = q
learningRate = 1e-8
for i in range(50):
    maxTemp,qTemp =update_q(qTemp,train_images,train_labels,learningRate)
    print(maxTemp)
    if  maxTemp>max:
        qMax =qTemp
        max = maxTemp
    # else: learningRate-= learningRate*0.2
    # if max-maxTemp>20:
    #     break


(10, 784)
51869
53449
53786
54040
54133
54242
54329
54343
54416
54474
54512
54528
54543
54534
54671
54665
54646
54764
54738
54736
54729
54779
54730
54771
54785
54850
54834
54827
54849
54867
54870
54915
54889
54868
54903
54919
54939
54894
54960


: 

thử với test

In [ ]:
a = accuracy(test_images[0:100,:],test_labels,qMax)
print(a)

thử với ảnh

In [ ]:
for i in range(100):
    pre  = predict(test_images[i].reshape(1,784),qMax)
    print(i)
    print("pre",pre)
    print("truth",np.argmax(test_labels[i]))
    plt.imshow(test_images[i,:].reshape(28,28))
    plt.show()